In [1]:
import os, glob
import cv2, sympy
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols
from sympy.matrices import *
from PIL import Image

In [2]:
Root = os.getcwd()
os.makedirs('Images_Folder', exist_ok=True)
os.makedirs('Result_Folder', exist_ok=True)

In [3]:
def Convert_to_PNG():
    os.chdir(os.path.join(Root,'Images_Folder'))
    image_patterns = ["*.jpeg", "*.jpg"]
    for pattern in image_patterns:
        for image_file in glob.glob(pattern):
            if not image_file.endswith(".png"):
                new_file = os.path.splitext(image_file)[0] + ".png"
                os.rename(image_file, new_file)
    os.chdir(Root)
Convert_to_PNG()

In [4]:
def Convert_Image_To_Matrix(Image_Name):
    if Image_Name in os.listdir('Images_Folder'):
        print("Found the picture:", Image_Name)
        Image = cv2.imread(os.path.join('Images_Folder', Image_Name))
        Gray_Image = np.zeros((Image.shape[0], Image.shape[1])) # Tạo ma trận 0 bằng kích thước với ma trận xuất ra từ hình ảnh
        for Row in range(Image.shape[0]): # chạy qua các hàng
            for Col in range(Image.shape[1]): # chạy qua các cột
                Pixel = Image[Row, Col]  # Lấy các giá trị Pixel của định dạng BRG
                Gray_Pixel = Pixel[0] * 0.114 + Pixel[1] * 0.587 + Pixel[2] * 0.299 # Chuyển sang Pixel của ảnh định dạng Gray
                Normalization = Gray_Pixel / 255 # Chuẩn hóa dữ liệu của mỗi Pixel bằng cách chia cho 255
                Gray_Image[Row, Col] = Normalization # Thêm các giá trị vào ảnh Gray
        return Gray_Image
    else: print("Picture not found!!!!")

In [5]:
def Create_Zero_Matrix(Values):
    Zero_Matrix = np.zeros((Values.shape[0], Values.shape[1]))
    return Zero_Matrix

def Create_Identity_Matrix(Matrix):
    Identity_Matrix = Create_Zero_Matrix(Matrix)
    for Row_Column in range(Identity_Matrix.shape[1]):
        Identity_Matrix[Row_Column,Row_Column] = 1
    return Identity_Matrix

In [6]:
def Find_Eigenvalue(Matrix):
    Identity_Matrix = Create_Identity_Matrix(Matrix)
    x = symbols('x') # Tạo ra biến tên là x thay cho λ để tính:  p(λ) = det(A - λI)
    Characteristic_Polynomial = Matrix - x * Identity_Matrix
    return sympy.solve(sympy.det(Characteristic_Polynomial))

def Find_Eigenvector(matrix):
  w,v = np.linalg.eig(matrix)
  return w,v

In [7]:
def Sort(Eigvaule_Vector):
    dict = {}
    for x in range(len(Eigvaule_Vector[0])):
        dict[Eigvaule_Vector[0][x]] = Eigvaule_Vector[1][:, x]
    key = list(dict.keys())
    key = sorted(key, key=None, reverse=True)
    return {i: dict[i] for i in key}

In [8]:
def Show_Image(Name, Result,k):
    os.chdir(os.path.join(Root, 'Result_Folder'))
    picture_name = Name
    img_array = np.array(Result, dtype=np.uint8)
    img = Image.fromarray(img_array)
    
    if picture_name in os.listdir():
        img = cv2.imread(picture_name)
        
    Filename = f"{k}_SVD Image Compression_2.jpg"
    
    if not os.path.exists(os.path.join(Root, 'Result_Folder')):
        os.makedirs(os.path.join(Root, 'Result_Folder'))
    
    cv2.imwrite(os.path.join(Root,'Result_Folder', Filename), img_array)

In [9]:
class Calculation:
    def __init__(self, Name, Matrix_Approximation):
        self.matrix = Convert_Image_To_Matrix(Name)
        self.shape = np.shape(self.matrix)
        self.k = Matrix_Approximation
        result = self.SVD()
        Show_Image(Name, result*255, self.k)


    def SVD(self):
      c,r = self.shape
      A = self.matrix
      V = []
      U = []
      print(np.shape(self.matrix))
      
      AtA = np.matmul(np.transpose(self.matrix),self.matrix)
      b = Find_Eigenvector(np.array(AtA,dtype= float))
      
      Newdict = Sort(b)

    # Tính UDV^T
      D = np.zeros(self.shape,dtype = 'complex_')
      r,c = self.shape
      for x in range(min(r,c)):
          D[x][x] = np.sqrt(np.abs(list(Newdict.keys())[x]))
      d = D[0:self.k,0:self.k]


      for x in Newdict.values():
          V.append(list(x))
      V = Matrix(V).T
      v = V[:,0:self.k]

      for i in range(min(self.k,r)):
        U.append((np.matmul(A,V[:,i])/D[i][i]).T[0])
      U = Matrix(U).T
      u = U[:, 0:self.k]

      a = u * d * (v.T)

      print('D = ',Matrix(d))
      print('V =',v)
      print('U = ',u)
      print('A =', a) 

      return a


if __name__ == '__main__':
    Calculation('Cute Cat.png', )


Found the picture: Cute Cat.png
(360, 360)
D =  Matrix([[158.245039593288, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 47.3845261260823, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 28.5748631399522, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 16.2169804774036, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 12.2051448685232, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 10.7091153116929, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 7.03844270129917, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 6.48820637569138, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 5.51671672003122, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 5.02466464405206]])
V = Matrix([[0.0312270530357368, 0.0174082904367910, 0.0286346753263125, 0.123702386106129, 0.0795227046202023, -0.0994942970757719, 0.0272618566728626, -0.0328424301942647, 0.0138212294276551, -0.0734535031842542], [0.0312911859376708, 0.0178192758778487, 0.0292235746156562, 0.122037224044394, 0.0782367400918407, -0.0986815342090838, 0.0271826105302911, -0.0328470680353527, 0.0149420958234174, -0.0745595003680583], [0.0312869203513666, 0.0180062690509681, 0.0297099666

In [10]:
os.chdir(Root)
os.getcwd()

'c:\\Users\\QuySech\\OneDrive\\Project\\Singular-Value-Decomposition'

In [10]:
import numpy as np

# Định nghĩa ma trận P
P = np.array([
    [0.06689, 0.0767, 0.0864],
    [0.335, 0.373, 0.418],
    [0.636, 0.694, 0.754]
])

# Tính eigenvalues và eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(P)

print("Eigenvalues:", eigenvalues)
eigenvectors


Eigenvalues: [ 1.20560426 -0.00123017 -0.0104841 ]


array([[-0.09841821, -0.47083932, -0.20082236],
       [-0.47783762,  0.81054406, -0.63637886],
       [-0.87291756, -0.34832264,  0.7447767 ]])